**네이버 토론방**

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from urllib.parse import parse_qs, urlparse
import pandas as pd

# User-Agent 설정
headers_pc = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/114.0.0.0 Safari/537.36"
}
headers_mobile = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X) "
                  "AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1"
}

def fetch_posts(stock_code, max_page=250):
    data = []
    for page in range(1, max_page + 1):
        url = f"https://finance.naver.com/item/board.naver?code={stock_code}&page={page}"
        resp = requests.get(url, headers=headers_pc)
        soup = BeautifulSoup(resp.text, "html.parser")
        post_links = soup.select("td.title a")

        for post_link in post_links:
            title = post_link.text.strip()
            relative_href = post_link["href"]
            qs = parse_qs(urlparse(relative_href).query)
            nid = qs.get("nid", [None])[0]
            if not nid:
                continue

            iframe_src = f"https://m.stock.naver.com/pc/domestic/stock/{stock_code}/discussion/{nid}"
            try:
                resp2 = requests.get(iframe_src, headers=headers_mobile)
                soup2 = BeautifulSoup(resp2.text, "html.parser")
                script_tag = soup2.find("script", id="__NEXT_DATA__")
                if not script_tag:
                    continue
                data_json = json.loads(script_tag.string)
                content_html = data_json["props"]["pageProps"]["dehydratedState"]["queries"][1]["state"]["data"]["result"]["contentHtml"]
                content_soup = BeautifulSoup(content_html, "html.parser")
                text = content_soup.get_text(separator="\n").strip()
                data.append({"제목": title, "본문": text})
            except:
                continue
    return data

if __name__ == "__main__":
    stock_code = input("종목 코드를 입력하세요: ").strip()
    posts = fetch_posts(stock_code, max_page=50) #페이지 단위로 추출
    df = pd.DataFrame(posts)
    output_file = f"naver_discussion_{stock_code}.csv"
    df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"CSV 파일로 저장 완료: {output_file}")

종목 코드를 입력하세요: ): 005930
CSV 파일로 저장 완료: naver_discussion_005930.csv


**네이버 증권 뉴스**

In [ ]:

import csv
import requests
from bs4 import BeautifulSoup
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# KoBERT 모델 로드
MODEL_NAME = "snunlp/KR-FinBERT-SC"
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME)
model.eval()

def fetch_news(stock_code, max_pages=10):
    all_news = []
    for page in range(1, max_pages + 1):
        url = f"https://finance.naver.com/item/news_news.naver?code={stock_code}&page={page}&sm=title_entity_id.basic&clusterId="
        headers = {
            "User-Agent": "Mozilla/5.0",
            "Referer": "https://finance.naver.com/"
        }
        resp = requests.get(url, headers=headers)
        soup = BeautifulSoup(resp.text, "html.parser")

        rows = soup.select("table.type5 tr")
        for row in rows:
            a_tag = row.select_one("td.title a")
            if a_tag:
                title = a_tag.get_text(strip=True)
                link = a_tag["href"]
                press = row.select_one("td.info")
                press_text = press.get_text(strip=True) if press else ""
                date = row.select_one("td.date")
                date_text = date.get_text(strip=True) if date else ""

                full_link = "https://finance.naver.com" + link
                all_news.append({
                    "날짜": date_text,
                    "언론사": press_text,
                    "제목": title,
                    "링크": full_link
                })
    return all_news

def fetch_article_content(url):
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://finance.naver.com"
    }
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")

    # 리다이렉트 처리
    redirect_script = soup.select_one("script")
    if redirect_script and "top.location.href" in redirect_script.get_text():
        try:
            redirect_url = redirect_script.get_text().split("top.location.href='")[1].split("'")[0]
            resp = requests.get(redirect_url, headers=headers)
            soup = BeautifulSoup(resp.text, "html.parser")
        except Exception:
            return "리다이렉트 실패"

    content_div = soup.select_one("#dic_area")
    if content_div:
        return content_div.get_text(separator="\n").strip()
    else:
        return "None"

def compute_sentiment_score(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1).squeeze()
    score = -1 * probs[0].item() + 0 * probs[1].item() + 1 * probs[2].item()
    return round(score, 4)

def save_news_with_sentiment(stock_code, max_pages=10, filename=f"naver_news_{stock_code}.csv"):
    news_list = fetch_news(stock_code, max_pages)
    for news in news_list:
        content = fetch_article_content(news["링크"])
        news["본문"] = content if content else "본문 없음"
        news["감성점수"] = compute_sentiment_score(content) if content else 0.0
        news.pop("언론사", None)
        news.pop("링크", None)

    fieldnames = ["날짜", "제목", "본문", "감성점수"]
    with open(filename, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for item in news_list:
            writer.writerow(item)

    print(f"{filename}")

if __name__ == "__main__":
    stock_code = input("종목 코드를 입력하세요: ").strip()
    save_news_with_sentiment(stock_code, max_pages=5) #페이지 단위로 추출


종목 코드를 입력하세요: 005930
naver_news_005930.csv


In [ ]:
!pip install newspaper3k
!pip install newspaper3k lxml[html_clean]

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 59.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 7.8 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=b8e9ef3f56c2495aa54d9663082b142b5b9dee109e34361c27db6cb011b544f5
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=870cf85271e6064d0c470d0c5eaa35227b4b29f4e60c0101bb547a28a7925cd4
  Stored in directory: /root/.cache/pip/wheels/80/d5/72/9cd9eccc819636436c6a6e59c22a0fb1ec

In [ ]:
!pip install transformers torch
import nltk
nltk.download('punkt_tab')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

**newsapi 해외뉴스**

In [ ]:
import requests
from datetime import datetime, timedelta
import pandas as pd
from newspaper import Article
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import re

# FinBERT 모델 초기화
def load_finbert_model():
    model_name = "yiyanghkust/finbert-tone"
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name)
    return tokenizer, model

# 텍스트 정제 함수
def clean_text(text):
    if not text:
        return ""
    text = text.lower()  # 소문자화
    text = re.sub(r'\s+', ' ', text)  # 다중 공백 제거
    text = re.sub(r'\n+', ' ', text)  # 줄바꿈 제거
    text = re.sub(r'[^a-zA-Z0-9.,!?\'\"%$()-]', ' ', text)  # 특수문자 제거
    text = text.strip()
    return text

# 감성 점수 계산 함수
#softmax: 각 클래스의 확률로 변환 (값의 총합 = 1)

def compute_sentiment_finbert(text, tokenizer, model):
    try:
        text = clean_text(text)
        tokens = tokenizer.encode(text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
        with torch.no_grad():
            output = model(tokens)
        logits = output.logits
        probs = torch.softmax(logits, dim=1).numpy()[0]
        sentiment_score = probs[2] - probs[0]
        return round(float(sentiment_score), 4)
    except Exception:
        return None

# 뉴스 수집 및 본문 크롤링 함수
def fetch_news_with_full_body_and_sentiment(ticker, days=3, max_results=100, api_key="5a9d9ae903084a349d94fdff1fb3da6a"):
    url = "https://newsapi.org/v2/everything"
    end_date = datetime.utcnow()
    start_date = end_date - timedelta(days=days)
    from_date_str = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")

    all_articles = []
    page = 1

    while True:
        params = {
            "q": ticker,
            "from": from_date_str,
            "language": "en",
            "sortBy": "publishedAt",
            "pageSize": 100,
            "page": page,
            "apiKey": api_key
        }

        response = requests.get(url, params=params)
        data = response.json()

        if data.get("status") != "ok":
            print(f"Error: {data.get('message')}")
            break

        articles = data.get("articles", [])
        if not articles:
            break

        all_articles.extend(articles)

        if len(all_articles) >= max_results or len(articles) < 100:
            break

        page += 1

    # 본문 크롤링 및 감성 점수 적용
    tokenizer, model = load_finbert_model()

    for article in all_articles:
        url = article.get("url")
        full_text = ""
        if url:
            try:
                news_article = Article(url)
                news_article.download()
                news_article.parse()
                full_text = news_article.text
            except Exception:
                full_text = article.get("description") or ""
        else:
            full_text = article.get("description") or ""

        article["full_body"] = full_text
        article["sentiment_score"] = compute_sentiment_finbert(full_text, tokenizer, model)

    newsapi = pd.DataFrame(all_articles)
    newsapi = newsapi[["publishedAt", "title", "full_body", "sentiment_score"]]
    newsapi["publishedAt"] = pd.to_datetime(newsapi["publishedAt"]).dt.strftime("%Y-%m-%d %H:%M:%S")
    newsapi.rename(columns={
        "publishedAt": "Date",
        "title": "Title",
        "full_body": "Body",
        "sentiment_score": "SentimentScore"
    }, inplace=True)

    newsapi = newsapi.head(max_results)

    start_str = start_date.strftime("%Y%m%d")
    end_str = end_date.strftime("%Y%m%d")
    filename = f"newsapi_{ticker}.csv"
    newsapi.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"{filename} 총 {len(newsapi)}건")

    return newsapi

if __name__ == "__main__":
    ticker_input = input("티커를 입력하세요: ").strip()
    fetch_news_with_full_body_and_sentiment(ticker_input)

티커를 입력하세요: AAPL


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


newsapi_AAPL.csv 총 45건


In [17]:
!pip install asyncpraw
!pip install nest_asyncio

**레딧** <br>
실행 후 티커가 아닌 검색어 입력

In [22]:
import asyncio
import nest_asyncio
import asyncpraw
from datetime import datetime, timedelta
import pandas as pd

nest_asyncio.apply()

reddit_api = asyncpraw.Reddit(
    client_id="4KYCTTNnmyNxyQwDS_fy6g",
    client_secret="CvxFEpg_IDU_uYxjJsrQWjOR2eXpnA",
    user_agent="stock_posts by /u/Leather_Arugula7011"
)

async def fetch_reddit_posts_async(ticker, subreddits=["stocks", "wallstreetbets", "investing"], days=3, limit=100):
    results = []
    now = datetime.utcnow()
    threshold_date = (now - timedelta(days=days)).date()
    time_threshold = datetime.combine(threshold_date, datetime.min.time())

    for subreddit_name in subreddits:
        subreddit = await reddit_api.subreddit(subreddit_name)
        async for submission in subreddit.search(ticker, sort="new", limit=limit):
            post_date = datetime.utcfromtimestamp(submission.created_utc)
            if post_date < time_threshold:
                continue

            results.append({
                "Subreddit": subreddit_name,
                "Date": post_date.strftime("%Y-%m-%d %H:%M:%S"),
                "Title": submission.title,
                "Body": submission.selftext[:500],
                "Link": f"https://www.reddit.com{submission.permalink}"
            })

    reddit_df = pd.DataFrame(results)
    filename = f"reddit_{ticker}.csv"
    reddit_df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"'{filename}' 총 {len(reddit_df)}건 ")

# Jupyter 환경에서 실행 예시
ticker = input("검색어를 입력하세요: ").strip()
await fetch_reddit_posts_async(ticker)


검색어를 입력하세요: Apple
'reddit_Apple.csv' 총 13건 
